In [ ]:
import requests
from typing import List
import chess
import chess.pgn
from io import StringIO
from neo4j import GraphDatabase
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

username = os.getenv('CHESSCOM_USERNAME')
email = os.getenv('CHESSCOM_EMAIL')
neo4j_uri = os.getenv('NEO4J_URI')
neo4j_user = os.getenv('NEO4J_USER')
neo4j_password = os.getenv('NEO4J_PASSWORD')

In [ ]:
def get_game_archives(): 
    headers = {'User-Agent': f'username: {username}, email: {email}'}
    URL = f'https://api.chess.com/pub/player/{username}/games/archives'

    data = requests.get(URL, headers=headers)
    if data.status_code == 200:
        return data.json().get('archives', [])
    else:
        print(f"Error: {data.status_code}")
        print(f"Response: {data.text}")
        return []

In [ ]:
def get_games_from_archive(archive_url: str) -> List[dict]:
    """
    Fetch all games from a specific monthly archive.
    
    Args:
        archive_url: URL of the monthly archive
        email: Your contact email for the User-Agent
        
    Returns:
        List of game dictionaries
    """
    headers = {'User-Agent': f'username: {username}, email: {email}'}
    
    response = requests.get(archive_url, headers=headers)
    
    if response.status_code == 200:
        return response.json().get('games', [])
    else:
        print(f"Error fetching {archive_url}: {response.status_code}")
        return []

In [72]:
def display_game_move_by_move(pgn_string: str):
    """
    Parse PGN and display the game move by move.
    
    Args:
        pgn_string: PGN text of the game
    """
    pgn = StringIO(pgn_string)
    game = chess.pgn.read_game(pgn)
    
    if not game:
        print("Failed to parse PGN")
        return
    
    print(f"White: {game.headers.get('White', 'Unknown')}")
    print(f"Black: {game.headers.get('Black', 'Unknown')}")
    print(f"Result: {game.headers.get('Result', '*')}")
    print(f"Date: {game.headers.get('Date', 'Unknown')}")
    print("\n" + "="*60 + "\n")
    

    board = game.board()
    move_number = 1
    
    for move in game.mainline_moves():
        if board.turn == chess.WHITE:
            print(f"{move_number}. {board.san(move)}", end=" ")
        else:
            print(f"{board.san(move)}")
            move_number += 1
        
        board.push(move)
    
    print("\n\n" + "="*60)
    print("Final position:")
    print(board)
    print(f"\nFEN: {board.fen()}")

if first_archive_games:
    display_game_move_by_move(first_archive_games[0]['pgn'])

White: TheBigWunk
Black: HrishabhD
Result: 1-0
Date: 2023.10.26


1. Nf3 d5
2. d4 Nf6
3. Bf4 e6
4. e3 Nc6
5. Nbd2 Bd6
6. Bxd6 Qxd6
7. Bd3 Qb4
8. O-O Qxb2
9. Ng5 e5
10. dxe5 Nxe5
11. Be2 Bg4
12. Bxg4 Nexg4
13. Qxg4 Nxg4
14. f3 Nxe3
15. Rae1 d4
16. Nc4 Qxc2
17. Nxe3 O-O
18. Nxc2 Rae8
19. Nxd4 Rxe1
20. Rxe1 f6
21. Nge6 Re8
22. Rd1 c5
23. Nxc5 b6
24. Na4 a6
25. Nxb6 Rd8
26. Nc4 a5
27. Nxa5 f5
28. Nac6 Rd6
29. a4 f4
30. a5 Rd5
31. a6 Ra5
32. a7 Ra6
33. Rb1 g5
34. Rb8+ Kf7
35. a8=Q Rxa8
36. Rxa8 Kf6
37. Rf8+ 

Final position:
. . . . . R . .
. . . . . . . p
. . N . . k . .
. . . . . . p .
. . . N . p . .
. . . . . P . .
. . . . . . P P
. . . . . . K .

FEN: 5R2/7p/2N2k2/6p1/3N1p2/5P2/6PP/6K1 b - - 2 37


In [ ]:
class ChessGraphBuilder:
    def __init__(self):
        """
        Initialize connection to Neo4j.
        

        """
        self.driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
    
    def close(self):
        self.driver.close()
    
    def add_game_to_graph(self, pgn_string: str, game_metadata: dict):
        """
        Parse a game and add all positions and moves to the graph.
        Routes to White or Black network based on player's color.
        
        Args:
            pgn_string: PGN text of the game
            game_metadata: Dictionary with game info for statistics
        """
        pgn = StringIO(pgn_string)
        game = chess.pgn.read_game(pgn)
        
        if not game:
            print("Failed to parse PGN")
            return
        
        # Determine if player was white or black
        player_color = None
        if game.headers.get('White', '').lower() == username.lower():
            player_color = 'white'
        elif game.headers.get('Black', '').lower() == username.lower():
            player_color = 'black'
        else:
            print(f"Player {username} not found in game")
            return
        
        # Determine outcome
        result = game.headers.get('Result', '*')
        if result == '1-0':
            outcome = 'win' if player_color == 'white' else 'loss'
        elif result == '0-1':
            outcome = 'win' if player_color == 'black' else 'loss'
        elif result == '1/2-1/2':
            outcome = 'draw'
        else:
            outcome = 'unknown'
        
        board = game.board()
        
        with self.driver.session() as session:
            # Start from initial position
            prev_fen = board.fen()
            move_number = 0
            
            # Create starting position node in appropriate network
            session.execute_write(self._create_position_node, prev_fen, player_color)
            
            # Process each move
            for move in game.mainline_moves():
                move_number += 1
                move_san = board.san(move)
                move_uci = move.uci()
                
                # Make the move
                board.push(move)
                current_fen = board.fen()
                
                # Create position node in appropriate network
                session.execute_write(self._create_position_node, current_fen, player_color)
                
                # Create/update move edge in appropriate network
                session.execute_write(
                    self._create_move_edge, 
                    prev_fen, 
                    current_fen, 
                    move_san,
                    move_uci,
                    outcome,
                    player_color
                )
                
                prev_fen = current_fen
            
            print(f"Added {player_color} game with {move_number} moves (outcome: {outcome})")
    
    @staticmethod
    def _create_position_node(tx, fen, player_color):
        """Create a Position node in the appropriate color network."""
        if player_color == 'white':
            query = """
            MERGE (p:WhitePosition {fen: $fen})
            RETURN p
            """
        else:
            query = """
            MERGE (p:BlackPosition {fen: $fen})
            RETURN p
            """
        tx.run(query, fen=fen)
    
    @staticmethod
    def _create_move_edge(tx, from_fen, to_fen, move_san, move_uci, outcome, player_color):
        """
        Create/update a MOVE relationship in the appropriate color network.
        """
        if player_color == 'white':
            query = """
            MATCH (from:WhitePosition {fen: $from_fen})
            MATCH (to:WhitePosition {fen: $to_fen})
            MERGE (from)-[m:MOVE {san: $move_san, uci: $move_uci}]->(to)
            ON CREATE SET 
                m.total_games = 1,
                m.wins = CASE WHEN $outcome = 'win' THEN 1 ELSE 0 END,
                m.losses = CASE WHEN $outcome = 'loss' THEN 1 ELSE 0 END,
                m.draws = CASE WHEN $outcome = 'draw' THEN 1 ELSE 0 END
            ON MATCH SET 
                m.total_games = m.total_games + 1,
                m.wins = m.wins + CASE WHEN $outcome = 'win' THEN 1 ELSE 0 END,
                m.losses = m.losses + CASE WHEN $outcome = 'loss' THEN 1 ELSE 0 END,
                m.draws = m.draws + CASE WHEN $outcome = 'draw' THEN 1 ELSE 0 END
            RETURN m
            """
        else:
            query = """
            MATCH (from:BlackPosition {fen: $from_fen})
            MATCH (to:BlackPosition {fen: $to_fen})
            MERGE (from)-[m:MOVE {san: $move_san, uci: $move_uci}]->(to)
            ON CREATE SET 
                m.total_games = 1,
                m.wins = CASE WHEN $outcome = 'win' THEN 1 ELSE 0 END,
                m.losses = CASE WHEN $outcome = 'loss' THEN 1 ELSE 0 END,
                m.draws = CASE WHEN $outcome = 'draw' THEN 1 ELSE 0 END
            ON MATCH SET 
                m.total_games = m.total_games + 1,
                m.wins = m.wins + CASE WHEN $outcome = 'win' THEN 1 ELSE 0 END,
                m.losses = m.losses + CASE WHEN $outcome = 'loss' THEN 1 ELSE 0 END,
                m.draws = m.draws + CASE WHEN $outcome = 'draw' THEN 1 ELSE 0 END
            RETURN m
            """
        
        tx.run(query,
               from_fen=from_fen,
               to_fen=to_fen,
               move_san=move_san,
               move_uci=move_uci,
               outcome=outcome)

In [ ]:
graph_builder = ChessGraphBuilder()

In [ ]:

archives = get_game_archives()

all_time_controls = {}

for archive_url in recent_archives:
    games = get_games_from_archive(archive_url)
    
    for game in games:
        tc = game.get('time_control', 'unknown')
        all_time_controls[tc] = all_time_controls.get(tc, 0) + 1


blitz_games = []

for archive_url in recent_archives:
    games = get_games_from_archive(archive_url, email)
    for game in games:
        if game.get('time_control') == '180+2':
            blitz_games.append(game)

for game in blitz_games:
    graph_builder.add_game_to_graph(game['pgn'], game)

Added white game with 43 moves (outcome: loss)
Added white game with 71 moves (outcome: win)
Added black game with 65 moves (outcome: loss)
Added black game with 92 moves (outcome: win)
Added white game with 70 moves (outcome: loss)
Added black game with 33 moves (outcome: loss)
Added white game with 68 moves (outcome: loss)
Added black game with 132 moves (outcome: win)
Added white game with 92 moves (outcome: loss)
Added white game with 105 moves (outcome: win)
Added black game with 43 moves (outcome: loss)
Added white game with 45 moves (outcome: win)
Added black game with 52 moves (outcome: win)
Added white game with 118 moves (outcome: loss)
Added black game with 20 moves (outcome: loss)
Added white game with 96 moves (outcome: loss)
Added black game with 76 moves (outcome: win)
Added white game with 118 moves (outcome: draw)
Added black game with 79 moves (outcome: loss)
Added white game with 40 moves (outcome: loss)
Added black game with 61 moves (outcome: loss)
Added white game